In [1]:
from configs import const

import os
import getpass
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough, ConfigurableField
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [2]:
os.environ["OPENAI_API_KEY"] = const.OPENAI_API_KEY
os.environ["NEO4J_URI"] = const.NEO4J_URI
os.environ["NEO4J_USERNAME"] = const.NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = const.NEO4J_PASSWORD

In [3]:
# 一度実行するとグラフがクラウド上に保存されるので2回目以降は実行しない。

# チャンク分けを行う（チャンクサイズ：512, オーバーラップ：125）
raw_documents = TextLoader(const.DATA_PATH + 'kaiji.txt').load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=125)
documents = text_splitter.split_documents(raw_documents)
documents[0]

Document(metadata={'source': '/Users/takuma.fukuda/Desktop/develop/mare-demo/data/kaiji.txt'}, page_content='賭博黙示録カイジ\n\n『賭博黙示録カイジ』（とばくもくしろくカイジ）は、福本伸行による日本の漫画。『週刊ヤングマガジン』（講談社）で1996年から連載された。\n\n続編として『賭博破戒録カイジ』（とばくはかいろくカイジ）、『賭博堕天録カイジ』（とばくだてんろくカイジ）、『賭博堕天録カイジ 和也編』、『賭博堕天録カイジ ワン・ポーカー編』が同誌に連載され、2017年からは『賭博堕天録カイジ 24億脱出編』と題して3勤1休のペースで連載している。なお、同誌目次では全シリーズ一貫して『カイジ』となっている。\n\n2019年6月時点でコミックスのシリーズ累計発行部数は2100万部を突破している[1]。\n\n本項では直接ストーリーが繋がっている続編であり、「賭博黙示録」と合わせて『カイジ』という一つの作品を')

In [5]:
# 一度実行するとグラフがクラウド上に保存されるので2回目以降は実行しない。

# グラフ形式のドキュメントに変換(結構時間がかかります)
# gpt-4oにおいて20分かかりました
llm=ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

# # neo4jのクラウド環境にアップロード
graph = Neo4jGraph()
# graph.add_graph_documents(
#     graph_documents,
#     baseEntityLabel=True,
#     include_source=True
# )

In [6]:
# 与えられたCypherクエリの結果のグラフを直接表示する。
# Cypherクエリは、グラフデータベース管理システムであるNeo4jで使用されるクエリ言語
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 10000"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

In [7]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [8]:
# グラフとドキュメントのハイブリットでvector_indexを構築
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [9]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [10]:
# Extract entities from text
class Entities(BaseModel):
    """エンティティに関する情報を識別するためのクラス"""

    names: List[str] = Field(
        ...,
        description="テキストに出現するすべての人物、組織、またはビジネスエンティティ",
    )

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたはテキストから組織および人物のエンティティを抽出します。",
        ),
        (
            "human",
            "以下の入力から情報を抽出するために、指定された形式を使用してください: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [12]:
entity_chain.invoke({"question": "カイジは豪遊した"}).names

['カイジ']

In [13]:
# 以下の3つの関数を繋ぐことでグラフ検索、類似文書検索を行う

def generate_full_text_query(input: str) -> str:
    """
    入力から曖昧検索を行い関連するエンティティを抜き出す。
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

def structured_retriever(question: str) -> str:
    """
    generate_full_text_queryで抜き出したエンティティを元に曖昧検索を行い、関連する情報を抜き出す。
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:20})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 1000
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

def retriever(question: str):
    """
    structured_retrieverで構造化データ（グラフ構造）から関連する情報を取得
    similarity_searchで構造化データ（一般的な類似度による検索）から情報を取得
    """
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
    {structured_data}
    Unstructured data:
    {"#Document ". join(unstructured_data)}
    """
    # print(final_data) # プロンプトを見たい時はこちらを実行
    return final_data

def retriever_only_unstructured(question: str):
    """
    similarity_searchで構造化データ（一般的な類似度による検索）から情報を取得
    """
    print(f"Search query: {question}")
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Unstructured data:
    {"#Document ". join(unstructured_data)}
    """
    # print(final_data) # プロンプトを見たい時はこちらを実行
    return final_data

In [14]:
# chainを作成する
template = """あなたは優秀なAIです。下記のコンテキストを利用してユーザーの質問に丁寧に答えてください。
必ず文脈からわかる情報のみを使用して回答を生成してください。
{context}

ユーザーの質問: {question}"""

prompt = ChatPromptTemplate.from_template(template)

def mkchain(retriever, chat_prompt, llm):
    """
    structured_retrieverで構造化データ（グラフ構造）から関連する情報を取得
    similarity_searchで構造化データ（一般的な類似度による検索）から情報を取得
    """
    # 引数として渡された関数を実行するためのラッパーを定義
    # 入力の"question"を取得する
    _search_query = RunnableLambda(lambda x: x["question"])
    chain = (
        RunnableParallel(
            {
                "context": _search_query | retriever, # _search_queryでquestionを取り出し、retrieverを実行
                "question": RunnablePassthrough(), # RunnablePassthrough入力を次のステップにそのまま渡す
            }
        )
        | prompt # プロンプトを作成
        | llm # LLMにプロンプトを渡して推論
        | StrOutputParser() # 出力はそのまま文字列で返す
    )
    return chain

# 実行結果

In [15]:
input_data = {"question": "カイジが嫌いな人は？"}

In [16]:
# Graph RAGの出力
chain = mkchain(retriever, prompt, llm)
result = chain.invoke(input_data)
print(result)

Search query: カイジが嫌いな人は？
カイジが嫌いな人については、文脈から以下の人物が挙げられます。

1. **兵藤会長** - カイジは兵藤会長に対して宣戦布告し、最終的には彼に敗北しています。また、兵藤会長はカイジにとって真に倒すべき存在とされています。
2. **利根川** - カイジは利根川と「Eカード」で対決し、極限の死闘を繰り広げました。
3. **村岡** - カイジは村岡を倒していますが、彼との対立がありました。
4. **和也** - カイジと和也は意見が対立し、激しい勝負を繰り広げました。

これらの人物はカイジにとって敵対的な存在であり、嫌いな人と考えられます。


In [17]:
# Graph RAGの最終的なプロンプトの確認
prompt_result = retriever(input_data["question"])
print(prompt_result)

Search query: カイジが嫌いな人は？
Structured data:
    カイジ - 制作 -> 日本テレビ
カイジ - 実写映画化 -> カイジ 人生逆転ゲーム
カイジ - ADAPTATION -> カイジ 人生逆転ゲーム
カイジ - ADAPTATION -> カイジ2 人生奪回ゲーム
カイジ - ADAPTATION -> カイジ ファイナルゲーム
カイジ - ADAPTATION -> 動物世界
カイジ - ADAPTATION -> カイジ 動物世界
カイジ - RECEIVED -> 講談社漫画賞
カイジ - INTERACTS_WITH -> 遠藤
カイジ - PARTICIPATES_IN -> 鉄骨渡り
カイジ - PARTICIPATES_IN -> ワン・ポーカー
カイジ - OWES_DEBT -> 帝愛
カイジ - GOES_TO -> エスポワール
カイジ - GOES_TO -> スターサイドホテル
カイジ - CHALLENGED -> 帝愛グループ
カイジ - CHALLENGED -> 兵藤
カイジ - CHALLENGED -> 利根川
カイジ - CHALLENGED -> ティッシュ箱くじ引き
カイジ - CHALLENGED -> 村岡
カイジ - DECLARED_WAR -> 兵藤会長
カイジ - LOST_TO -> 兵藤会長
カイジ - LOST_AT -> スターサイドホテル
カイジ - REQUESTED -> 遠藤
カイジ - FORCED_LABOR -> 帝愛グループ
カイジ - BELONGS_TO -> E班
カイジ - DECEIVED -> 遠藤
カイジ - DECEIVED -> 帝愛
カイジ - MEMBER_OF -> E班
カイジ - MEMBER_OF -> 45組
カイジ - DEFEATED -> 大槻
カイジ - DEFEATED -> 村岡
カイジ - DEFEATED -> 和也
カイジ - OBTAIN -> 6000万円
カイジ - POSSESS -> 80万円
カイジ - LEAVE -> 地下
カイジ - DURATION -> 20日間
カイジ - SEARCH -> 裏カジノ
カイジ - MEET -> 坂崎
カイジ - MEET -> チャン

In [18]:
# RAGの出力
chain = mkchain(retriever_only_unstructured, prompt, llm)
result = chain.invoke(input_data)
print(result)

Search query: カイジが嫌いな人は？
カイジが嫌いな人について、文脈からわかる情報を基にお答えします。

カイジが嫌いな人物として挙げられるのは、帝愛グループの会長・兵藤です。カイジは「鉄骨渡り」や「Eカード」などの極限のギャンブルを通じて、兵藤会長が真に倒すべき存在であることを痛感しています。また、地下労働施設での班長・大槻もカイジにとって敵対する存在であり、彼の巧みな篭絡やイカサマによりカイジは苦しめられています。

したがって、カイジが嫌いな人としては、兵藤会長と大槻が挙げられます。


In [22]:
# RAGの最終的なプロンプトの確認
prompt_result = retriever_only_unstructured(input_data["question"])
print(prompt_result)

Search query: カイジが嫌いな人は？
Unstructured data:
    
text: 人間の思考、生き様が描かれており、作品独自のギャンブルと、「ざわ‥ざわ‥」の擬音やモブキャラの「黒服」などの福本作品独自の表現が特徴である。

元々は前後編の読み切りの予定だったが、福本がヤングマガジン編集部に限定ジャンケンのプロットを話したところで連載が決まり[2][出典無効]、その後、福本の最大のヒット作品になった。本作の大ヒットにより、それまで麻雀漫画家というイメージの強かった福本の名は一般にも大きく知られるようになった。

映像作品ではテレビアニメが『逆境無頼カイジ』のタイトルで日本テレビで制作され、2007年10月に第1シーズンが、2011年4月から第2シーズンが放送された。また、2009年10月には『カイジ 人生逆転ゲーム』のタイトルで実写映画化もされた。映画は2011年11月に『カイジ2 人生奪回ゲーム』のタイトルでシリーズ2作目が公開（いずれも日本#Document 
text: れる。土壇場で裏切るのが人間の真実だと主張する和也と、必ずしもそんな人間ばかりではないはずだとそれを否定するカイジは激しく意見が対立。その言い合いを契機に、友情確認ゲーム「救出」による人間性の実験、カイジと和也の勝負が始まった。

「救出」に挑むのは日本人の光山、中国人のチャン、フィリピン人のマリオのアジア3人組。3人は幾度となくピンチを迎えながらも成功を重ねるが、疑念と保身、友愛と自己愛を巡る死闘の末、光山の裏切りによって「救出」は終了。 その後、敗者となったチャンとマリオの処刑が行われる寸前、和也の元から思わず処刑実行ボタンのついたリモコンを奪いとったカイジは自身の判断で勝手に処刑の中止ボタンを押してしまう。しかしそのボタンだけでは処刑は止まらぬ仕組みになっており処刑を止めるためには暗証番�#Document 
text: ��で強制労働をさせられることになった。カイジは一日外出券を得るために金を貯めようとするが、所属するE班の班長・大槻の巧みな篭絡により金を使い果たす。大槻はさらにカイジに給料を前貸しし、自身の主催する「地下チンチロリン」に誘い込む。大槻に大敗を喫してさらなる借金生活に追い込まれるも、カイジは大槻のイカサマに気付き、自分と同じ境遇にある通称「4